In [1]:
from GSA import grounded_sam_simple_demo
from GSA import automatic_label_simple_demo
import metal_seg
import gpt4v
import json

name_img='83.jpg'

final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
contents=gpt4v.llm_getobj(name_img)
content = json.loads(contents)

{
"Type": ["banana", "canned food"],
"Color": ["yellow", "gold"],
"Material": ["plastic", "aluminum"],
"Grasping method": ["gripping", "suction cup"],
"Placement status": ["not", "not"]
}


In [3]:
items = content["Type"]
colors = content["Color"]
materials=content["Material"]
# obj_list =['banana', 'red carton', 'spray bottle', 'clamp', 'scissors', 'wrench', 'canned food', 'key']
print(items)
print(colors)
print(materials)
obj_list = [f"{color} {item}" for item, color in zip(items, colors)]
# 创建一个空列表来存储匹配的索引
metal_indices = []

# 遍历 materials 列表
for index, material in enumerate(materials):
    # 检查 material 是否包含 "metal"
    if "metal" in material:
        # 如果包含，将索引添加到 matching_indices 列表中
        metal_indices.append(index)

# 打印匹配的索引
print(metal_indices)
print(obj_list)


['banana', 'canned food']
['yellow', 'gold']
['plastic', 'aluminum']
[]
['yellow banana', 'gold canned food']


In [4]:
# obj_list = ['silver metal']
# obj_list = ['yellow banana', 'red and white canned food 1', 'red carton', 'blue and yellow canned food 2', 'blue and white spray bottle', 'black clamp', 'gray and orange scissors', 'silver wrench', 'silver key']

In [5]:
contours_all=grounded_sam_simple_demo.ground_sam(name_img,obj_list)
metal_contours,mask=metal_seg.metal_seg(metal_indices)

Before NMS: 3 boxes
After NMS: 2 boxes
labels= ['yellow banana 0.91', 'gold canned food 0.57']
(2, 1279, 1707)


In [8]:
import numpy as np
import cv2
for i in range(len(contours_all)):
    # print("class_name[i]:",class_name[i])
    # 轮廓质心计算
    contours_i=contours_all[i]
    contours_i=np.vstack(contours_i)
    contour_2d=np.array(contours_i).reshape(-1, 2)
    # print("contour_2d:",contour_2d)
    M = cv2.moments(contour_2d)
    x_center = int(M["m10"] / M["m00"])
    y_center = int(M["m01"] / M["m00"])
    print("x_center:",x_center,"y_center:",y_center)
    img = np.zeros((720, 1280, 3), dtype=np.uint8)
    img.fill(255)  # 以白色填充
    mu00 = M["m00"]
    mu11 = M["m11"] - x_center*M["m01"]
    mu20 = M["m20"] - x_center*M["m10"]
    mu02 = M["m02"] - y_center*M["m01"]
    theta = 1/2*np.arctan2(2*mu11/mu00, (mu20 - mu02)/mu00)
    print("theta(deg)=",theta/np.pi*180)



x_center: 1025 y_center: 485
theta(deg)= 15.292797960705427
x_center: 545 y_center: 686
theta(deg)= 33.19265224851114
x_center: 1182 y_center: 868
theta(deg)= 7.902650004030668
